# Curso Big Data #5 - Transformaciones y operaciones en PySpark

#### 1. start a Spark session

In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('pyspark_exercises').getOrCreate()

#### 2. Crear el dataframe 

In [7]:
df=spark.read.csv('C:/Users/pc/pruebas/walmart_stock.csv', inferSchema=True, header=True)

#### 3. Mostrar los nombres de las columnas

In [8]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### 4. Mostrar info

In [9]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

#### 5. Como es el schema?

In [10]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### 6. Imprimir las 5 primeras columnas

In [12]:
df.select(df.columns[:5]).show()

+-------------------+------------------+------------------+------------------+------------------+
|               Date|              Open|              High|               Low|             Close|
+-------------------+------------------+------------------+------------------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18|
|2012-01-10 00:00:00|             59.43|59.709998999999996|             58.98|59.040001000000004|
|2012-01-11 00:00:00|         59.060001|         59.529999|59.040001000000004|         59.400002|
|2012-01-12 00:00:00

#### 7. Usamos describe() para aprender acerca del DataFrame.

In [14]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [15]:
## solo con dos decimales

des = df.describe()

des.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [16]:
from pyspark.sql.functions import format_number

In [18]:
des.select('summary',
           format_number(des['Open'].cast('Float'), 2).alias('Open'),
           format_number(des['High'].cast('Float'), 2).alias('High'),
           format_number(des['Low'].cast('Float'), 2).alias('Low'),
           format_number(des['Close'].cast('Float'), 2).alias('Close'),
           format_number(des['Volume'].cast('Float'), 2).alias('Volume'),
           format_number(des['Adj Close'].cast('Float'), 2).alias('Adj Close')).show()

+-------+--------+--------+--------+--------+-------------+---------+
|summary|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+--------+--------+--------+--------+-------------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|    84.91|
+-------+--------+--------+--------+--------+-------------+---------+



#### 8. Cree un nuevo dataframe con una columna llamada Relación HV que es la relación entre el precio alto y el volumen de acciones negociadas en un día.

In [19]:
df.columns

new_df = df.withColumn('HV Ratio', df['High']/df['Volume'])

In [21]:
new_df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|            HV Ratio|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51

#### 9. ¿Qué día tuvo el precio máximo máximo?

In [23]:
df.sort('High').show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2015-11-13 00:00:00|56.740002000000004|         57.060001|         56.299999|         56.419998|12514900|53.975581000000005|
|2015-11-02 00:00:00|57.290001000000004|57.610001000000004|             56.77|57.610001000000004|10719200|         55.114026|
|2015-10-28 00:00:00|             57.66|         57.720001|             57.16|57.639998999999996|11834700|         55.142725|
|2015-11-12 00:00:00|57.639998999999996|             57.77|         56.919998|         56.950001| 9551500|54.482620999999995|
|2015-11-16 00:00:00|56.389998999999996|         58.029999|56.360001000000004|         57.869999|13321600|55.362759999

In [24]:
df.orderBy(df['High'].desc()).show()

+-------------------+-----------------+-----------------+-----------------+-----------------+--------+-----------------+
|               Date|             Open|             High|              Low|            Close|  Volume|        Adj Close|
+-------------------+-----------------+-----------------+-----------------+-----------------+--------+-----------------+
|2015-01-13 00:00:00|        90.800003|        90.970001|            88.93|        89.309998| 8215400|        83.825448|
|2015-01-08 00:00:00|        89.209999|90.66999799999999|            89.07|        90.470001|12713600|84.91421600000001|
|2015-01-09 00:00:00|            90.32|        90.389999|            89.25|        89.349998| 8522500|        83.862993|
|2015-01-12 00:00:00|        89.360001|        90.309998|        89.220001|        90.019997| 7372500|        84.491846|
|2015-01-23 00:00:00|88.41999799999999|        89.260002|        87.889999|        88.510002| 7565800|83.07458100000001|
|2015-01-26 00:00:00|        88.

In [25]:
df.orderBy(df['High'].desc()).head(1)[0]['Date']

datetime.datetime(2015, 1, 13, 0, 0)

#### 10. ¿Cuál es la media de la columna Cerrar?

In [26]:
from pyspark.sql.functions import mean

In [27]:
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### 11. ¿Cuál es el máximo y el mínimo de la columna Volumen?

In [28]:
from pyspark.sql.functions import max, min
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### 12. ¿Cuántos días estuvo close por debajo de los 60 dólares?

In [29]:
df[df['Close'] < 60].count()

81

#### 13. ¿Qué porcentaje de veces el máximo superó los 80 dólares?
#### En otras palabras, (Número de días máximo>80)/(Total de días en el conjunto de datos)

In [30]:
df[df['High'] > 80].count() / df.count() * 100

9.141494435612083

#### 14. ¿Cuál es la correlación de Pearson entre Alto y Volumen?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [32]:
df.corr('High', 'Volume')

-0.3384326061737161

#### 15. ¿Cuál es el máximo máximo por año?

In [34]:
from pyspark.sql.functions import year

In [35]:
df_year = df.withColumn('Year', year('Date'))

In [36]:
df_year.groupBy('Year').max().select('Year', 'max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### 16. ¿Cuál es el cierre promedio para cada mes calendario?
#### En otras palabras, a lo largo de todos los años, ¿cuál es el precio de cierre promedio para enero, febrero, marzo, etc.? Su resultado tendrá un valor para cada uno de estos meses.

In [37]:
from pyspark.sql.functions import month, col

df_month = df.withColumn('Month', month('Date'))
df_month.groupBy('Month').avg().select('Month', col('avg(Close)').alias('Average Close')).show()

+-----+-----------------+
|Month|    Average Close|
+-----+-----------------+
|   12|72.84792478301885|
|    1|71.44801958415842|
|    6| 72.4953774245283|
|    3|71.77794377570092|
|    5|72.30971688679247|
|    9|72.18411785294116|
|    4|72.97361900952382|
|    8|73.02981855454546|
|    7|74.43971943925233|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|    2|  71.306804443299|
+-----+-----------------+

